# 1. Introduction and Background
Describe what your notebook discusses and give some background information.
Be sure to include an "Eye Catching" graphic in the beginning of the notebook.

# 2. Examples
Teach the content you mentioned in your introduction. Use interactive widgets and animations to help "teach by example". Where applicable also use Python/R examples.

# 3. Include questions 
throughout/in its own section in order to engage the student/make it easier for a teacher to use your notebook.

# 4. Conclusion

# 5. Reiterate important learning objectives 
that you covered and explain where this knowledge is helpful